Adrian Wiegman, 2023-08-07

In this notebook I calculate groundwater depths from modflow simulated water table elevations and lidar surface elevation data. 

In [1]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook

# Create new file geodatabase to store results
gdb = "gwdepth.gdb"
# only run this once
#if not os.path.exists(os.path.join(wdr,gdb)):
#arcpy.management.CreateFileGDB(wdr,gdb)
ap.env.workspace = os.path.join(wdr,gdb)

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinfo2', '$name')
File:      c:\windows\system32\<ipython-input-1-28f96385f735>
Type: 

In [ ]:
# copy lidar data (surface elevation meters above sea level) to the geodatabase

lid_m = arcpy.sa.Raster(r"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10m")
lid_m.save("lid_elev_meters")

In [ ]:
# copy the raw contour data into the project geodatabase

In [43]:
# plymouth carver ---
arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\Massachusetts\USGS_\Plymouth_Carver_Water_Tables_2ft_Contours\Plymouth_Carver_Water_Tables_2ft_Contours.shp",
    out_feature_class="Water_Tables_2ft_Contours_Plymouth_Carver")

arcpy.management.CalculateField(
    in_table=r"C:\Workspace\Geodata\MEP\gwdepth.gdb\Water_Tables_2ft_Contours_Plymouth_Carver",
    field="ELEV_METER",
    expression="!Contour!*0.3048",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.DeleteField(
    in_table="Water_Tables_2ft_Contours_Plymouth_Carver",
    drop_field="Contour",
    method="DELETE_FIELDS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwdepth.gdb\\Water_Tables_2ft_Contours_Plymouth_Carver'>

In [44]:
# cape cod ---
arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\Massachusetts\USGS_\Cape_Cod_Water_Tables_2ft_Contours\Cape_Cod_Water_Tables_2ft_Contours.shp",
    out_feature_class="Water_Tables_2ft_Contours_Cape_Cod_Raw")

arcpy.management.DeleteField(
    in_table="Water_Tables_2ft_Contours_Cape_Cod_Raw",
    drop_field="ELEV_FTASL;CONTOUR;Shape_Leng",
    method="DELETE_FIELDS"
)

# removing the first four features since they are located on mainland (not on the cape)
selection = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Water_Tables_2ft_Contours_Cape_Cod_Raw",
    selection_type="NEW_SELECTION",
    where_clause="OBJECTID NOT IN (1, 2, 3, 4)",
    invert_where_clause=None
)

# save the edited layer
arcpy.management.CopyFeatures(
    in_features=selection,
    out_feature_class="Water_Tables_2ft_Contours_Cape_Cod")

<Result 'C:\\Workspace\\Geodata\\MEP\\gwdepth.gdb\\Water_Tables_2ft_Contours_Cape_Cod'>

In [45]:
# Merge the cape cod and plymouth carver water table contour lines
arcpy.management.Merge(
    inputs="Water_Tables_2ft_Contours_Cape_Cod;Water_Tables_2ft_Contours_Plymouth_Carver",
    output="Water_Tables_2ft_Contours_Merge",
    add_source="ADD_SOURCE_INFO"
)

arcpy.management.DeleteField(
    in_table="Water_Tables_2ft_Contours_Merge",
    drop_field="OBJECTID",
    method="DELETE_FIELDS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwdepth.gdb\\Water_Tables_2ft_Contours_Merge'>

In [58]:
tin_file = r"C:\Workspace\Geodata\MEP\outputs\tin_gw_meters"

In [ ]:
# NOTE I USE AN EDITED version of the contour lines layer that adds a vertex at 21.5m elevation on the cape.

In [59]:
# Create TIN (triangular irregular network) from merged contour lines and convert to raster
arcpy.ddd.CreateTin(
    out_tin=tin_file,
    in_features="Water_Tables_2ft_Contours_Merge_Edit ELEV_METER Hard_Line <None>",
    constrained_delaunay="DELAUNAY"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\outputs\\tin_gw_meters'>

In [62]:
with arcpy.EnvManager(snapRaster="lid_elev_meters"):
    arcpy.ddd.TinRaster(
        in_tin=tin_file,
        out_raster=r"C:\Workspace\Geodata\MEP\gwdepth.gdb\gw_elev_meters",
        data_type="FLOAT",
        method="LINEAR",
        sample_distance="CELLSIZE",
        z_factor=1,
        sample_value=10
    )

In [4]:
# copy the mask of the groundwater contour lines to the gdb
arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\MEP\outputs\Contour_Mask.gdb\Placemarks\Polygons",
    out_feature_class=r"contour_mask_poly")

<Result 'C:\\Workspace\\Geodata\\MEP\\gwdepth.gdb\\contour_mask_poly'>

In [5]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', snapRaster="gw_elev_meters", extent='346823.303957774 4597731.43403507 422133.303957774 4662251.43403507 PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', cellSize="gw_elev_meters"):
    arcpy.conversion.FeatureToRaster(
        in_features=r"Contour_Mask\Polygons",
        field="OID",
        out_raster="contour_mask_rast")

In [6]:
mask = arcpy.sa.Raster("contour_mask_rast")

In [7]:
gw_elev_meters = arcpy.sa.Raster("gw_elev_meters")*mask
lid_elev_meters = arcpy.sa.Raster("lid_elev_meters")
gw_depth_meters = lid_elev_meters - gw_elev_meters
gw_depth_meters.save("gw_depth_meters")

In [8]:
gw_depth_lt1_meters = gw_depth_meters < 1
gw_depth_lt1_meters.save("gw_depth_lt_1m")

In [9]:
gw_depth_meters = arcpy.sa.Raster("gw_depth_meters")
gw_depth_lt1_meters = gw_depth_meters < 0.5
gw_depth_lt1_meters.save("gw_depth_lt_0p5m")

In [10]:
gw_elev_meters.save("gw_elev_meters_masked")